In [2]:
# Import the two main libraries
import datetime
import json
import pickle
import sys
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup  # to process html
from tqdm import tqdm

In [3]:
"""
dtypes = {
    "link": "category",
    "title": "string",
    "author": "string",
    "date": "string",
    "text": "string",
}

articles = pd.read_csv("FalsoArticles.csv", index_col=0, dtype=dtypes)
"""

In [ ]:
articels = pd.read_parquet("FalsoArticles.parquet.snappy", engine="fastparquet")

In [3]:
articles

,link,title,author,date,text
0,https://www.ilfattoquotidiano.it/2023/05/17/da...,“Da esponente del ‘partito unico bellicista’ v...,Thomas Mackinson,17 Maggio 2023,La controffensiva di Kiev? “Deve ancora cominc...
1,https://www.ilfattoquotidiano.it/2023/05/16/ar...,‘Armi offensive? Sarebbe una guerra mondiale’....,Gianni Rosini,16 Maggio 2023,"Da mesi, ormai, la guerra mostra un sostanzial..."
2,https://www.ilfattoquotidiano.it/2023/05/14/oc...,"“Occupare città russe, bombardare oleodotti e ...",F. Q.,14 Maggio 2023,Occupare città russe per guadagnare vantaggio ...
3,https://www.ilfattoquotidiano.it/2023/05/14/ir...,Irritazione della Santa Sede dopo il no di Zel...,Francesco Antonio Grana,14 Maggio 2023,“Con le armi non si otterrà mai la sicurezza e...
4,https://www.ilfattoquotidiano.it/2023/05/14/ze...,Zelensky dice no alla mediazione di Papa Franc...,F. Q.,14 Maggio 2023,Una chiusura che certifica lo stallo. Le spera...
...,...,...,...,...,...
2981,https://www.ilfattoquotidiano.it/2022/02/22/uc...,"Ucraina, Draghi: “Da mosca inaccettabile viola...",F. Q.,22 Febbraio 2022,“Voglio esprimere la mia più ferma condanna pe...
2982,https://www.ilfattoquotidiano.it/2022/02/22/uc...,"Ucraina, fuochi d’artificio e festeggiamenti a...",F. Q.,22 Febbraio 2022,Un piccolo gruppo di persone ha festeggiato sv...
2983,https://www.ilfattoquotidiano.it/2022/02/22/uc...,"Ucraina, mezzi militari russi entrano nella re...",F. Q.,22 Febbraio 2022,Mosca ha annunciato il riconoscimento dell’ind...
2984,https://www.ilfattoquotidiano.it/2022/02/21/uc...,"Ucraina, Putin riconosce il Donbass: la firma ...",Agenzia Vista Alexander Jakhnagiev,21 Febbraio 2022,Alla presenza dei due leader delle repubbliche...


In [4]:
articles["comments"] = 0
articleNumber = []

for i, link in tqdm(enumerate(articles.link)):
    comments = []

    articleNumber = link.split("/")[-2]

    data = requests.get(
        "https://comments.ilfattoquotidiano.it/?p="
        + str(articleNumber)
        + "&orderby=upvote"
    )
    data = json.loads(data.text)

    for comment in enumerate(data["comments"]):
        comments.append(comment[1]["content"])

    articles["comments"][i] = comments

0it [00:00, ?it/s]/tmp/ipykernel_467498/125774678.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles["comments"][i] = comments
/home/tosk/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
2986it [11:21,  4.38it/s]


In [8]:
articles.to_parquet("fatto.parquet.snappy", engine="fastparquet")